## imports

In [1]:
{-# LANGUAGE PackageImports #-}
{-# LANGUAGE OverloadedStrings #-}
import "atidot-anonymizer" Atidot.Anonymizer.Monad
import qualified "atidot-anonymizer" Atidot.Anonymizer.Run as Anonymizer (runExample)
import "atidot-anonymizer" Atidot.Anonymizer.XML
import "atidot-anonymizer" Atidot.Anonymizer.Types
import "atidot-anonymizer" Atidot.Anonymizer.Utils
import "mtl"          Control.Monad.State (evalStateT)
import "data-default" Data.Default (def)

import Control.Monad
import qualified Data.Text as T
import qualified Data.ByteString.Lazy as BL
import qualified "bytestring" Data.ByteString.Lazy.Char8 as BL8


## Anonymization Paths

In [2]:
anonymizationPaths :: [Path]
anonymizationPaths = [["/","TXLife","TXLifeRequest" ,"OLifE","Holding", "Policy", "PolNumber"]
                    ,["/","TXLife","TXLifeRequest" ,"OLifE","Holding", "Policy","OLifEExtension", "JournalEntry", "EntryAuthor"]
                    ,["/","TXLife","TXLifeRequest" ,"OLifE","Holding", "Investment", "Banking"]
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"FullName"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"GovtID"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Person", "FirstName"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Person", "MiddleName"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Person", "LastName"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Person", "Prefix"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Person", "Suffix"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Address", "Line1"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Address", "Line2"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Address", "Line3"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Address", "Line4"]    
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"Phone" ]
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "Party" ,"EMailAddress" ]
                    ,["/","TXLife","TXLifeRequest" ,"OLifE", "FinancialStatement", "PolicyStatement", "PolNumber"]
                    ]

xmlPath = "acord.xml"

## Example XML:
let's print the example xml used in this notebook

In [4]:
xmlData <- BL.readFile xmlPath
BL8.putStrLn $ BL.take 100 xmlData

<interactive>:1:16: error:
    • Couldn't match expected type ‘BL.ByteString’ with actual type ‘[a0]’
    • In the second argument of ‘($)’, namely ‘take 100 xmlData’
      In the expression: BL8.putStrLn $ take 100 xmlData
      In an equation for ‘it’: it = BL8.putStrLn $ take 100 xmlData
<interactive>:1:25: error:
    • Couldn't match expected type ‘[a0]’ with actual type ‘BL.ByteString’
    • In the second argument of ‘take’, namely ‘xmlData’
      In the second argument of ‘($)’, namely ‘take 100 xmlData’
      In the expression: BL8.putStrLn $ take 100 xmlData

## Hashed Version

In [5]:
accordHashingScript :: Anonymizer ()
accordHashingScript = do
    whileNext $ do
        path <- current
        when (path `elem` anonymizationPaths) hash
    return ()    

execute `runExample` from Atidot.Anonymizer.Run, that prints two things:
1. XML after the script's traversal on the file
1. The paths traversed by the script

In [6]:
Anonymizer.runExample xmlPath accordHashingScript

<?xml version="1.0" encoding="UTF-8"?>
<TXLife xmlns="http://ACORD.org/Standards/Life/2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://ACORD.org/Standards/Life/2 file:///C:/Documents%20and%20Settings/azl6pgm/Desktop/TXLife2.25.00.xsd">
   <TXLifeRequest>
      <TransRefGUID>0226201108301200180001</TransRefGUID>
      <TransType tc="1203">Holding Transmittal</TransType>
      <TransExeDate>2011-08-30</TransExeDate>
      <TransExeTime>12:11:03.0000000-06:00</TransExeTime>
      <TestIndicator tc="0">False</TestIndicator>
      <OLifE>
         <SourceInfo>
            <CreationDate>2011-08-30</CreationDate>
            <OLifEExtension VendorCode="33">
               <EventType>Policy</EventType>
               <EventName>Add Rider</EventName>
               <EventCode>133</EventCode>
               <MessageVersion>1.0</MessageVersion>
               <MessageVersionDate>2011-10-26</MessageVersionDate>
               <BatchID>1</BatchID>
            </OL